In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math as math
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import Matern, RationalQuadratic,ExpSineSquared,PairwiseKernel
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.gaussian_process.kernels import (StationaryKernelMixin,NormalizedKernelMixin,Kernel,Hyperparameter,squareform,pdist,cdist)
import scipy.spatial.distance as distance
import joblib
import matplotlib.pyplot as plt
import os

In [14]:
#columns = ["x","y","z","w","data","error"]
datatab = pd.read_table(r"C:\Users\seanw\OneDrive\Desktop\GitHub\FYP\Data\SEOB_7D_MM_data_chi_par.txt")
# Normalize the 'Mass' column by dividing by 150
datatab['Mass'] = datatab['Mass'] / 150.0  # This scales Mass to [0.25, 1]

# Rename for consistency with the rest of your code
datatab.columns = ['w', 'q', 'chi1', 'chi2', 'theta1', 'theta2', 'phis', 'mismatch', 'error']
datatab

w = datatab['w']
q = datatab['q']
chi1 = datatab['chi1']
chi2 = datatab['chi2']
theta1 = datatab['theta1']
theta2 = datatab['theta2']
phis = datatab['phis']
mismatch = datatab['mismatch']
error = datatab['error']


# First Mass
datam1 = datatab[0:250]
wm1 = datam1['w']
qm1 = datam1['q']
chi1m1 = datam1['chi1']
chi2m1 = datam1['chi2']
theta1m1 = datam1['theta1']
theta2m1 = datam1['theta2']
phism1 = datam1['phis']
mismatchm1 = datam1['mismatch']
em1 = datam1['error']
X1 = np.column_stack((wm1,qm1,chi1m1,chi2m1,theta1m1,theta2m1,phism1,mismatchm1))

# Second Mass
datam2 = datatab[250:500]
wm2 = datam2['w']
qm2 = datam2['q']
chi1m2 = datam2['chi1']
chi2m2 = datam2['chi2']
theta1m2 = datam2['theta1']
theta2m2 = datam2['theta2']
phism2 = datam2['phis']
mismatchm2 = datam2['mismatch']
em2 = datam2['error']
X2 = np.column_stack((wm2, qm2, chi1m2, chi2m2, theta1m2, theta2m2, phism2, mismatchm2))

# Third Mass
datam3 = datatab[500:750]
wm3 = datam3['w']
qm3 = datam3['q']
chi1m3 = datam3['chi1']
chi2m3 = datam3['chi2']
theta1m3 = datam3['theta1']
theta2m3 = datam3['theta2']
phism3 = datam3['phis']
mismatchm3 = datam3['mismatch']
em3 = datam3['error']
X3 = np.column_stack((wm3, qm3, chi1m3, chi2m3, theta1m3, theta2m3, phism3, mismatchm3))

# Fourth Mass
datam4 = datatab[750:]
wm4 = datam4['w']
qm4 = datam4['q']
chi1m4 = datam4['chi1']
chi2m4 = datam4['chi2']
theta1m4 = datam4['theta1']
theta2m4 = datam4['theta2']
phism4 = datam4['phis']
mismatchm4 = datam4['mismatch']
em4 = datam4['error']
X4 = np.column_stack((wm4, qm4, chi1m4, chi2m4, theta1m4, theta2m4, phism4, mismatchm4))


X = np.column_stack((w,q,chi1,chi2,theta1,theta2,phis))
D = np.concatenate((mismatchm1,mismatchm2,mismatchm3,mismatchm4))
E = np.concatenate((em1,em2,em3,em4))

In [15]:
indices = np.arange(X.shape[0])
X_trainval, X_test, D_trainval, D_test, E_trainval, E_test, indices_trainval, indices_test = train_test_split(X, D,E,indices, test_size=0.1, random_state=42)

In [18]:
bounds_lmult = (1e-4,10) # Fix l1 bounds
guess_lmult = (1,1,1,1,1,1,1)
bounds_lsingle = (1e-4, 10.0)
ExpSinelbounds = (1e-2, 10.0)
guess_lsingle = 1.0
guess_signal_var = 1.0
bounds_signal_var = (1e-2, 20)
nu = 1.75
nu2 = 0.75
alpha = 1
periodicity = 2
periodictybounds = (1, 20)
noise_init_noerr = 1
noise_bounds_noerr = (1e-6, 1e6)
gamma = 1
E_var = E_trainval**2
##### bounds for noise level decided by the range(5,95)% of error. Tighter than no_error but not as tight as mean. Allowing for the model to have a decent bit of variation
p5, p95 = np.percentile(E_var, [5, 95])
noise_init_minmaxerr = np.mean(E_var)
noise_bounds_minmaxerr = (p5, p95)


RBFMat = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=(2,2,2,2,2,2,2), length_scale_bounds=(1,100)) + C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * Matern(length_scale=(0.5,0.5,0.5,0.5,0.5,0.5,0.5), length_scale_bounds=(1e-2,10), nu=.75)
RBFLaplace = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=(1,1,1,1,1,1,1), length_scale_bounds=(0.1,100)) + C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * PairwiseKernel(gamma=1, gamma_bounds = (0.5,10),metric='laplacian')
Mat_noerr = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * Matern(length_scale=guess_lmult, length_scale_bounds=bounds_lmult, nu=nu) + WhiteKernel(noise_level=noise_init_noerr, noise_level_bounds=noise_bounds_noerr)
Laplace_noerr = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * PairwiseKernel(gamma=gamma, gamma_bounds=(0.1,50), metric='laplacian') + WhiteKernel(noise_level=noise_init_noerr, noise_level_bounds=noise_bounds_noerr)
RBF_noerr = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=guess_lmult, length_scale_bounds=(1e-2,10)) + WhiteKernel(noise_level=noise_init_noerr, noise_level_bounds=noise_bounds_noerr)
Mat_minmaxerr = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * Matern(length_scale=guess_lmult, length_scale_bounds=bounds_lmult, nu=nu) + WhiteKernel(noise_level=noise_init_minmaxerr, noise_level_bounds=noise_bounds_minmaxerr)
Laplace_minmaxerr = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * PairwiseKernel(gamma=gamma, gamma_bounds=(0.1,50), metric='laplacian') + WhiteKernel(noise_level=noise_init_minmaxerr, noise_level_bounds=noise_bounds_minmaxerr)
RBF_minmaxerr = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=guess_lmult, length_scale_bounds=bounds_lmult) + WhiteKernel(noise_level=noise_init_minmaxerr, noise_level_bounds=noise_bounds_minmaxerr)

# Kernels = {
#     "RBFMat": RBFMat,
#     "RBFLap": RBFLaplace,
#     "Mat_noerr": Mat_noerr,
#     "Laplace_noerr": Laplace_noerr,
#     "RBF_noerr": RBF_noerr,
#     "Mat_minmaxerr": Mat_minmaxerr,
#     "Laplace_minmaxerr": Laplace_minmaxerr,
#     "RBF_minmaxerr": RBF_minmaxerr
# }

Kernels = {"RBFLap":RBFLaplace}

base_dir = r'C:\Users\seanw\OneDrive\Desktop\GitHub\FYP\Final_gps'

for kernel_name, kernel_obj in tqdm(Kernels.items(), desc="Training and saving GPR models"):
    gpr = GaussianProcessRegressor(kernel=kernel_obj, normalize_y=True, n_restarts_optimizer=2, random_state=42)
    gpr.fit(X_trainval, D_trainval)
    # Save the model
    model_filename = os.path.join(base_dir, f'7DGPR_{kernel_name}.joblib')
    joblib.dump(gpr, model_filename)


Training and saving GPR models:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified lower bound 0.1. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__constant_value is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__gamma is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
Training and sav